# Trabajo práctico 1 (Parte 1.2) - Organización de datos
### Facultad de ingeniería - Universidad de Buenos Aires


---


Alumna: Agustina Bocaccio \\
Padron: 106393 \\
Email: abocaccio@fi.uba.ar \\

---

#### Segundo cuatrimestre 2021
#### Fecha de entrega: 14/10

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Parte 1.2

### 46) Si decimos que la ubicación de una categoría es el promedio de la latitud y longitud de sus contenidos geolocalizados que son miembros de ella (si es que tiene): ¿Cuales son las dos categorías más cercanas?

In [ ]:
id='12KNPiAF8AsGTLGRSpMUw2XdwYXwSosr3'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('geo_tags.csv')

geo_tags = pd.read_csv('geo_tags.csv') 

In [ ]:
id='1-JktDI4xrc3EDsnDUvsMqUFw67Art3_M'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('category_links.csv')

category_links = pd.read_csv('category_links.csv', usecols=['cl_from', 'cl_to']) 

In [ ]:
category_links

,cl_from,cl_to
0,5,Wikipedia:Mantenimiento
1,5,Wikipedia:PÃ¡ginas_con_enlaces_mÃ¡gicos_de_ISBN
2,7,Andorra
3,7,Instituciones_patrocinadoras_de_equipos_ciclistas
4,7,Wikipedia:ArtÃ­culos_con_datos_locales
...,...,...
13308614,10010781,Nacidos_en_1994
13308615,10010781,Personas_vivas
13308616,10010783,Fundaciones_en_SudÃ¡frica
13308617,10010787,Fundaciones_en_SudÃ¡frica_por_aÃ±o


In [ ]:
geo_tags

,gt_id,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,158041,4328020,earth,0,41.507500,-5.817500,1000,NaN,NaN,NaN,NaN
1,5844377,4670424,earth,0,13.738000,-89.292000,10000,city,NaN,NaN,NaN
2,5844381,4670424,earth,0,13.702000,-89.208000,10000,city,NaN,NaN,NaN
3,5844383,4670424,earth,0,13.400000,-89.750000,10000,city,NaN,NaN,NaN
4,5844388,4670424,earth,0,12.850000,-88.000000,10000,city,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
556388,136672179,5239183,earth,0,3.450000,-76.533333,1000,NaN,NaN,NaN,NaN
556389,136672180,104468,earth,1,30.815839,45.996069,1000,NaN,NaN,NaN,NaN
556390,136672181,968281,earth,1,46.953333,6.256944,10000,city,NaN,NaN,NaN
556391,136672182,79985,earth,0,45.109659,7.641221,500,NaN,NaN,NaN,NaN


In [ ]:
geo_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556393 entries, 0 to 556392
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   gt_id       556393 non-null  int64  
 1   gt_page_id  556393 non-null  int64  
 2   gt_globe    556393 non-null  object 
 3   gt_primary  556393 non-null  int64  
 4   gt_lat      556393 non-null  float64
 5   gt_lon      556393 non-null  float64
 6   gt_dim      556393 non-null  int64  
 7   gt_type     449893 non-null  object 
 8   gt_name     72557 non-null   object 
 9   gt_country  79192 non-null   object 
 10  gt_region   3088 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 46.7+ MB


In [ ]:
pd.DataFrame({'Cantidad de nulos': geo_tags.isnull().sum()})

,Cantidad de nulos
gt_id,0
gt_page_id,0
gt_globe,0
gt_primary,0
gt_lat,0
gt_lon,0
gt_dim,0
gt_type,106500
gt_name,483836
gt_country,477201


In [ ]:
pd.DataFrame({'Cantidad de nulos': category_links.isnull().sum()})

,Cantidad de nulos
cl_from,0
cl_to,0


Los dataframes no tienen valores nulos en las columnas que afectan a mi problema, por lo que no es necesario limpiarlos

Agrupo por id del contenido al que corresponde el geo_tags y aplico el promedio a la latitud y a la longitud.

In [ ]:
geo_mean = geo_tags.loc[:,['gt_page_id','gt_lat', 'gt_lon']].groupby('gt_page_id').agg({'gt_lat': 'mean', 'gt_lon': 'mean'})
geo_mean

,gt_lat,gt_lon
gt_page_id,,
7,42.500000,1.500000
10,-34.599722,-58.381944
15,42.506111,1.522222
26,43.162222,-2.070000
39,34.532778,69.165833
...,...,...
10010686,23.250000,91.450000
10010689,34.341944,62.203056
10010690,23.000000,91.730000


In [ ]:
geo_categories = pd.merge(left=category_links, right=geo_tags, left_on='cl_from', right_on='gt_page_id')
geo_categories

,cl_from,cl_to,gt_id,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,7,Andorra,136647726,7,earth,0,42.500000,1.500000,10000,city,NaN,NaN,NaN
1,7,Instituciones_patrocinadoras_de_equipos_ciclistas,136647726,7,earth,0,42.500000,1.500000,10000,city,NaN,NaN,NaN
2,7,Wikipedia:ArtÃ­culos_con_datos_locales,136647726,7,earth,0,42.500000,1.500000,10000,city,NaN,NaN,NaN
3,7,Wikipedia:ArtÃ­culos_con_datos_por_trasladar_a...,136647726,7,earth,0,42.500000,1.500000,10000,city,NaN,NaN,NaN
4,7,Wikipedia:ArtÃ­culos_con_enlaces_externos_rotos,136647726,7,earth,0,42.500000,1.500000,10000,city,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042005,10010690,Wikipedia:ArtÃ­culos_con_datos_por_trasladar_a...,136668617,10010690,earth,1,23.000000,91.730000,10000,city,NaN,NaN,NaN
3042006,10010690,Wikipedia:ArtÃ­culos_con_ficha_sin_actualizar,136668617,10010690,earth,1,23.000000,91.730000,10000,city,NaN,NaN,NaN
3042007,10010690,Wikipedia:ArtÃ­culos_sin_imagen_en_entidad_sub...,136668617,10010690,earth,1,23.000000,91.730000,10000,city,NaN,NaN,NaN
3042008,10010724,RÃ­os_de_Nueva_Gales_del_Sur,136670269,10010724,earth,1,-30.931944,147.628889,1000,landmark,NaN,NaN,NaN


In [ ]:
# agrupo por id del contenido al que corresponde el geo_tags y aplico el promedio a la latitud y a la longitud.
geo_mean = geo_categories.loc[:,['cl_to','gt_lat', 'gt_lon']].groupby('cl_to').mean()
a = np.array(geo_mean[['gt_lat', 'gt_lon']].values.tolist())
a
geo_mean

,gt_lat,gt_lon
cl_to,,
1._FC_Colonia,50.933611,6.875000
1._FC_Heidenheim_1846,48.668500,10.139300
1._FC_Kaiserslautern,49.434444,7.775833
1._FC_NÃ¼rnberg,49.426389,11.125278
1._FC_Union_Berlin,52.457222,13.568056
...,...,...
Å koda,50.421100,14.930600
Å»agaÅ,51.616667,15.316667
Å»nin,52.849580,17.719920


In [ ]:
from sklearn.neighbors import KDTree
tree = KDTree(a)
dist, ind = tree.query(a, k=2)                
print(ind)  # indices of 2 closest neighbors
print(dist)  # distances to 2 closest neighbors

[[     0  80614]
 [     1  10951]
 [ 30997      2]
 ...
 [108039  86958]
 [108040 105397]
 [108041  91688]]
[[0.         0.00644806]
 [0.         0.01694943]
 [0.         0.        ]
 ...
 [0.         0.0660249 ]
 [0.         0.05390422]
 [0.         0.03146806]]


In [ ]:
min_index= np.argmin(dist[:,1])
print(ind[min_index])
print(a[ind[min_index][0]], a[ind[min_index][1]])

[30997     2]
[49.434444  7.775833] [49.434444  7.775833]


In [ ]:
print('Categorias mas cercanas:')
print(geo_categories.loc[ind[min_index][0], ['cl_to']])
print(geo_categories.loc[ind[min_index][1], ['cl_to']])

Categorias mas cercanas:
cl_to    Wikipedia:PÃ¡ginas_que_utilizan_Timeline
Name: 30997, dtype: object
cl_to    Wikipedia:ArtÃ­culos_con_datos_locales
Name: 2, dtype: object


### 53) Si para un usuario tenemos la cantidad de acciones que realizó para cada tipo de log y la cantidad de veces que fue bloqueado: ¿Cuál es la acción que más y menos correlaciona con ser bloqueado? ¿Qué acción correlaciona más con saber algo(babel>=0) de inglés?

In [ ]:
id='1E1HtTbXVdwJJW7Z7XeyMf9H-GQVgD0OK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('logs.csv')

In [ ]:
id='1RILGHMiAuB3n3Nc1n1cHshOR4QkQLDmb'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('languages.csv')

languages = pd.read_csv('languages.csv')
languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35666 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35666 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35666 non-null  object
dtypes: int64(1), object(2)
memory usage: 836.0+ KB


Me quedo solo con las columnas que uso

In [ ]:
logs = pd.read_csv('logs.csv', usecols=['contributor_ip', 'contributor_id' ,'contributor_username','title', 'action']) 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
pd.DataFrame({'Cantidad de nulos': logs.isnull().sum(), 'Porcentaje de nulos': (logs.isnull().mean() * 100)})

,Cantidad de nulos,Porcentaje de nulos
contributor_username,125253,1.027595
contributor_id,125253,1.027595
contributor_ip,12064770,98.981241
action,2,0.000016
title,2005,0.016449


In [ ]:
logs

,contributor_username,contributor_id,contributor_ip,action,title
0,Sabbut,2075.0,NaN,delete,Verificacionismo
1,Sabbut,2075.0,NaN,delete,Rapunzel
2,Sabbut,2075.0,NaN,delete,Macromedia Flash
3,Sanbec,7856.0,NaN,delete,Archivo:Gnu head.png
4,Sanbec,7856.0,NaN,delete,Archivo:Gnu-head-sm.jpg
...,...,...,...,...,...
12188941,SeroBOT,4980693.0,NaN,create,Usuario discusión:El guachicol
12188942,Leonpolanco,561850.0,NaN,create,Usuario discusión:2806:2F0:3140:BE0:C09B:D1C4:...
12188943,Desiretjs,6335019.0,NaN,create,Usuario:Desiretjs
12188944,Correogsk,443260.0,NaN,create,Contenedores


Busco los usuarios que fueron bloqueados, y luego veo las acciones que realizaron

In [ ]:
blocks = logs.loc[logs[('action')]=="block",:] #filtro solo los bloqueos
blocked_users = blocks['title'].str.replace("Usuario:", "")
blocked_users

5             212.55.30.157
287          201.241.228.15
288           201.255.50.32
290            84.122.0.158
313            81.192.12.93
                 ...       
12188865    186.141.200.121
12188886          Galaxi066
12188909        Xx.tkpro.xx
12188924    181.167.104.152
12188930      190.18.98.138
Name: title, Length: 342302, dtype: object

In [ ]:
blocked_username_actions = (logs.loc[logs['contributor_username'].isin(blocked_users), :])
blocked_username_actions 

,contributor_username,contributor_id,contributor_ip,action,title
0,Sabbut,2075.0,NaN,delete,Verificacionismo
1,Sabbut,2075.0,NaN,delete,Rapunzel
2,Sabbut,2075.0,NaN,delete,Macromedia Flash
3,Sanbec,7856.0,NaN,delete,Archivo:Gnu head.png
4,Sanbec,7856.0,NaN,delete,Archivo:Gnu-head-sm.jpg
...,...,...,...,...,...
12188905,Latothy99,6001164.0,NaN,thank,Usuario:JulioW4rrior00
12188913,NaN,NaN,190.122.184.86,create,Chiquimula F.C.
12188915,NaN,NaN,2806:108E:1C:366A:EC17:C8DB:35:3EAF,create,Black Hawk Down (libro)
12188918,NaN,NaN,186.73.165.140,create,Bastón Sonajero Guna


Cuento las acciones de los usuarios bloqueados y me fijo cuales son las que mas y menos se repiten

In [ ]:
blocked_actions = blocked_username_actions.loc[:, ['action']].value_counts()
print('Accion que mas se relaciona con ser bloqueado: '+ blocked_actions.idxmax()[0])
print('Accion que menos se relaciona con ser bloqueado: '+ blocked_actions.idxmin()[0])

Accion que mas se relaciona con ser bloqueado: delete
Accion que menos se relaciona con ser bloqueado: profilesave


In [ ]:
languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35666 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35666 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35666 non-null  object
dtypes: int64(1), object(2)
memory usage: 836.0+ KB


In [ ]:
languages.loc[languages['babel_level'] ==  "N", 'babel_level'] = 1 
pd.to_numeric(languages['babel_level'])
english_speakers = (languages.loc[(languages['babel_lang'] == "en") & ((languages['babel_level']).astype("int") >= 1), :])['babel_user']
english_speakers

3906         311
3907        3712
3908        3823
3909        5639
3910        7856
          ...   
13355    6253125
13356    6254900
13357    6277026
13358    6327824
13359    6331490
Name: babel_user, Length: 9454, dtype: int64

In [ ]:
actions_english_speakers = (logs.loc[logs['contributor_id'].isin(english_speakers), :])
actions_english_speakers

,contributor_username,contributor_id,contributor_ip,action,title
0,Sabbut,2075.0,NaN,delete,Verificacionismo
1,Sabbut,2075.0,NaN,delete,Rapunzel
2,Sabbut,2075.0,NaN,delete,Macromedia Flash
3,Sanbec,7856.0,NaN,delete,Archivo:Gnu head.png
4,Sanbec,7856.0,NaN,delete,Archivo:Gnu-head-sm.jpg
...,...,...,...,...,...
12188928,Leonpolanco,561850.0,NaN,create,Usuario discusión:190.18.98.138
12188932,Leonpolanco,561850.0,NaN,create,Usuario discusión:190.110.109.44
12188937,Leonpolanco,561850.0,NaN,create,Usuario discusión:2800:810:4FF:A95:D5AA:C859:F...
12188942,Leonpolanco,561850.0,NaN,create,Usuario discusión:2806:2F0:3140:BE0:C09B:D1C4:...


In [ ]:
actions_english_speakers = actions_english_speakers.loc[:, ['action']].value_counts()
print('Accion que mas se relaciona con saber ingles: '+ actions_english_speakers.idxmax()[0])
print('Accion que menos se relaciona con saber ingles: '+ actions_english_speakers.idxmin()[0])

Accion que mas se relaciona con saber ingles: delete
Accion que menos se relaciona con saber ingles: new
